In [13]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [14]:
# This should be updated
root_dir = '/Users/lucas/Data_Science_Portfolio/M&M project/M&M_images_cropped'

validation_dir = os.path.join(root_dir, 'validation')
training_dir = os.path.join(root_dir, 'training')

In [4]:
os.makedirs(validation_dir)
os.makedirs(training_dir)

os.makedirs(os.path.join(training_dir, 'M&Ms'))
os.makedirs(os.path.join(training_dir, 'Not_M&Ms'))

os.makedirs(os.path.join(validation_dir, 'M&Ms'))
os.makedirs(os.path.join(validation_dir, 'Not_M&Ms'))

In [14]:
m_and_ms = os.path.join(root_dir, 'M&Ms')

split_size = 0.8
number_of_m_and_ms = len(os.listdir(m_and_ms))
number_of_training = round(number_of_m_and_ms*split_size)
m_and_ms_training = os.path.join(training_dir, 'M&Ms')
m_and_ms_validation = os.path.join(validation_dir, 'M&Ms')
                         
for i in os.listdir(m_and_ms)[:number_of_training]:
    if(os.path.getsize(os.path.join(m_and_ms, i))):
      copyfile(os.path.join(m_and_ms, i), os.path.join(m_and_ms_training, i))
                                 
for j in os.listdir(m_and_ms)[number_of_training:]:
    if(os.path.getsize(os.path.join(m_and_ms, i))):
        copyfile(os.path.join(m_and_ms, j), os.path.join(m_and_ms_validation, j))

In [15]:
nots = os.path.join(root_dir, 'Not_M&Ms')

not_training = os.path.join(training_dir, 'Not_M&Ms')
not_validation = os.path.join(validation_dir, 'Not_M&Ms')
number_of_nots = len(os.listdir(nots))
number_of_training = round(number_of_nots*split_size)

for i in os.listdir(nots)[:number_of_training]:
    if(os.path.getsize(os.path.join(nots, i))):
        copyfile(os.path.join(nots, i), os.path.join(not_training, i))
                       
for j in os.listdir(nots)[number_of_training:]:
    if(os.path.getsize(os.path.join(nots, j))):
        copyfile(os.path.join(nots, j), os.path.join(not_validation, j))

In [15]:
train_datagen = ImageDataGenerator(rescale= 1/255)

train_generator = train_datagen.flow_from_directory(directory=training_dir,
                                                   batch_size=64,
                                                   class_mode='binary',
                                                   target_size=(100, 100))

validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                              batch_size=64,
                                                              class_mode='binary',
                                                              target_size=(100,100))

Found 24 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


In [16]:
from tensorflow.keras.optimizers import RMSprop

model = tf.keras.models.Sequential([ 
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
    
    
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
history = model.fit(train_generator, 
                    epochs=15, 
                    verbose=1, 
                    validation_data=validation_generator)

Epoch 1/15


2022-11-27 18:04:12.968467: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-27 18:04:13.259358: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 786ms/step - loss: 0.6788 - accuracy: 0.6667 - val_loss: 3.2410 - val_accuracy: 0.6667
Epoch 2/15
1/1 [==============================] - 0s 78ms/step - loss: 3.0168 - accuracy: 0.6250 - val_loss: 8.5753 - val_accuracy: 0.3333
Epoch 3/15


2022-11-27 18:04:13.693038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 69ms/step - loss: 7.2518 - accuracy: 0.3750 - val_loss: 1.2961 - val_accuracy: 0.3333
Epoch 4/15
1/1 [==============================] - 0s 67ms/step - loss: 1.2384 - accuracy: 0.3750 - val_loss: 0.5801 - val_accuracy: 0.6667
Epoch 5/15
1/1 [==============================] - 0s 64ms/step - loss: 0.5185 - accuracy: 0.6667 - val_loss: 0.4207 - val_accuracy: 0.8333
Epoch 6/15
1/1 [==============================] - 0s 64ms/step - loss: 0.5375 - accuracy: 0.7083 - val_loss: 1.2535 - val_accuracy: 0.6667
Epoch 7/15
1/1 [==============================] - 0s 66ms/step - loss: 0.7346 - accuracy: 0.6250 - val_loss: 0.3714 - val_accuracy: 0.8333
Epoch 8/15
1/1 [==============================] - 0s 62ms/step - loss: 0.4813 - accuracy: 0.7917 - val_loss: 0.3146 - val_accuracy: 0.6667
Epoch 9/15
1/1 [==============================] - 0s 61ms/step - loss: 0.2073 - accuracy: 0.9583 - val_loss: 0.1723 - val_accuracy: 1.0000
Epoch 10/15
1/1 [=====================

In [18]:
import pickle

with open('history.pkl', 'wb') as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://44e4a438-9561-46b0-863c-5ec4bb33b781/assets


INFO:tensorflow:Assets written to: ram://44e4a438-9561-46b0-863c-5ec4bb33b781/assets
